# Gün 4 bölüm 1: Salgın verileriyle gerçek zamanlı

<div style="background: #f8f9fa; padding: 0.5rem 1rem; border-radius: 8px; margin-bottom: 1rem; border-left: 4px solid #007bff;">
<a href="../../index.ipynb" style="text-decoration: none; color: #007bff; font-weight: bold;">← Kurs İndeksine Geri Dön</a> | 
<a href="../../index-tr.ipynb" style="text-decoration: none; color: #dc3545; font-weight: bold;">🇹🇷 Türkçe İndeks</a>
</div>

---

## Giriş

Bu uygulama (iki bölümde) bir Ebola Virüs Hastalığı (EVD) salgınının erken değerlendirmesini ve yeniden yapılandırılmasını simüle eder. Vaka ölüm oranı (CFR), epieğriler, büyüme oranı tahmini, temas izleme verileri, gecikmeler ve bulaştırıcılık tahminleri dahil olmak üzere salgının erken aşamasının analizinin çeşitli yönlerini tanıtır.

## Öğrenme çıktıları

Bu uygulamanın sonunda şunları yapabilmelisiniz:

- R'de salgın verilerini yükleme ve temizleme ([bölüm 1](./real-time-response-1.html))

- Vaka ölüm oranını (CFR) tahmin etme ([bölüm 1](./real-time-response-1.html))

- Çizgi listesinden insidans hesaplama ve çizme ([bölüm 1](./real-time-response-1.html))

- Salgının büyüme oranını ve ikiye katlanma süresini tahmin etme ve yorumlama ([bölüm 2](./real-time-response-2.html))

- Bulaştıran / bulaşan birey çiftlerine dair verilerden seri aralığı tahmin etme ([bölüm 2](./real-time-response-2.html))

- Salgının üreme sayısını tahmin etme ve yorumlama ([bölüm 2](./real-time-response-2.html))

- Kısa vadeli gelecekteki insidansı öngörme ([bölüm 2](./real-time-response-2.html))


## Batı Afrika'da kurgusal bir ülkede yeni bir EVD salgını

Batı Afrika'da kurgusal bir ülkede yeni bir EVD salgını bildirildi. Sağlık Bakanlığı salgın müdahalesini koordine etmekle görevli ve sizi gerçek zamanlı müdahaleyi bilgilendirmek için salgın analizi konusunda danışman olarak görevlendirdi.

## Gerekli paketler 

CRAN veya github'da mevcut olan aşağıdaki paketler bu analiz için gereklidir.
Gerekli paketleri şu şekilde kurun:

Paketler kurulduktan sonra yeni bir R oturumu açmanız gerekebilir. Ardından kütüphaneleri şu şekilde yükleyin:

In [ ]:
library(readxl)
library(outbreaks)
library(incidence)
library(distcrete)
library(epitrix)
library(EpiEstim)
library(projections)
library(ggplot2)
library(magrittr)
library(binom)
library(ape)
library(outbreaker2)
library(here)

## Erken veriler (verileri R'ye okuma)

Size aşağıdaki çizgi listesi ve temas verileri verildi:

**linelist_20140701.xlsx**: 1 Temmuz 2014'e kadar vaka bilgilerini içeren bir çizgi listesi; ve

**contact_20140701.xlsx**: 1 Temmuz 2014'e kadar vakalar arasında bildirilen temasların listesi. "infector" potansiyel enfeksiyon kaynağını, "case_id" ise temasın alıcısını belirtir.

R'ye okumak için bu dosyaları indirin ve verileri içe aktarmak için `readxl` paketindeki `read_xlsx()` fonksiyonunu kullanın. Her içe aktarma işlemi `tibble` nesnesi olarak saklanan bir veri tablosu oluşturacak.

- Birincisini `linelist`, ve  
- ikincisini `contacts` olarak adlandırın. 

Örneğin, ilk komut satırınız şöyle görünebilir:  



In [ ]:
linelist <- read_excel(here("data","linelist_20140701.xlsx"), na = c("", "NA"))
contacts <- read_excel(here("data","contacts_20140701.xlsx"), na = c("", "NA"))

Buradaki verilere ve yapıya bakmak için biraz zaman ayırın.

- Veriler ve format daha önce gördüğünüz çizgi listelerine benziyor mu?
- Salgın araştırma ekibinin bir parçası olsaydınız, başka hangi bilgileri toplamak isteyebilirdiniz?

In [ ]:
dim(linelist)
head(linelist)

- Ayrıca rapor tarihi, yaş, hanehalkı tanımlayıcısı, meslek vb. veriler toplamak isteyebilirsiniz.

Daha ileri analizler için tüm tarihlerin `Date` nesneleri olarak doğru şekilde saklandığından emin olmanız gerektiğini unutmayın. Bunu `as.Date` fonksiyonunu kullanarak yapabilirsiniz, örneğin:

In [ ]:
linelist$date_of_onset <- as.Date(linelist$date_of_onset, format = "%Y-%m-%d")

In [ ]:
linelist$date_of_infection <- as.Date(linelist$date_of_infection, format = "%Y-%m-%d")
linelist$date_of_hospitalisation <- as.Date(linelist$date_of_hospitalisation, format = "%Y-%m-%d")
linelist$date_of_outcome <- as.Date(linelist$date_of_outcome, format = "%Y-%m-%d")

Biçimlendirilmiş veriler şöyle görünmelidir:

In [ ]:
head(linelist)
head(contacts)

## Veri temizleme ve tanımlayıcı analiz

Bu `linelist`te bulunan verilere daha yakından bakın.

- Ne fark ediyorsunuz?

In [ ]:
head(linelist)
names(linelist)

Eksik girişler olduğunu fark edebilirsiniz.
Analizde önemli bir adım, veri girişindeki hataları belirlemektir.
Hastane isimlerindeki hataları değerlendirmek zor olsa da, enfeksiyon tarihinin her zaman semptom başlangıcı tarihinden önce olmasını bekleriz.
  
Negatif veya 0 günlük kuluçka dönemleri olan girişleri kaldırmak için bu veri setini temizleyin.

In [ ]:
## veri girişindeki hataları belirle (negatif kuluçka dönemi)
mistakes <- which(linelist$date_of_onset <= linelist$date_of_infection)
mistakes
linelist[mistakes, ]


In [ ]:
## veri girişindeki hataları belirle (negatif kuluçka dönemi)
mistakes <- which(linelist$date_of_onset <= linelist$date_of_infection)
mistakes
linelist[mistakes, ] # negatif veya 0 kuluçka süresi olan sadece ilk birkaç girişi göster.

"Temizlenmiş" çizgi listenizi yeni bir nesne olarak kaydedin: `linelist_clean`

In [ ]:
linelist_clean <- linelist[-mistakes, ]

Tam veri setiniz olsaydı kontrol etmek isteyebileceğiniz başka hangi negatif tarihler veya hatalar olabilir?

## Vaka ölüm oranının (CFR) hesaplanması

İşte sonuç durumuna göre vaka sayıları. Bundan CFR'yi nasıl hesaplarsınız? 

In [ ]:
table(linelist_clean$outcome, useNA = "ifany")

Sonucu NA olan vakalarla ne yapacağınızı düşünün?

In [ ]:
n_dead <- sum(linelist_clean$outcome %in% "Death")
n_known_outcome <- sum(linelist_clean$outcome %in% c("Death", "Recover"))
n_all <- nrow(linelist_clean)
cfr <- n_dead / n_known_outcome
cfr_wrong <- n_dead / n_all

cfr_with_CI <- binom.confint(n_dead, n_known_outcome, method = "exact")
cfr_wrong_with_CI <- binom.confint(n_dead, n_all, method = "exact")

print(cfr_with_CI)
print(cfr_wrong_with_CI)

## İnsidans eğrilerine bakış

Bilmek istediğimiz ilk soru basitçe: ne kadar kötü?. Analizin ilk adımı tanımlayıcıdır - bir salgın eğrisi veya epieğri çizmek istiyoruz. Bu, semptom başlangıcı tarihine göre zaman içindeki insidansı görselleştirir.

`incidence` paketini kullanarak, semptom başlangıcı tarihlerine dayalı olarak `linelist_clean`den günlük insidansı hesaplayın. Sonucu i_daily adlı bir nesnede saklayın; sonuç şöyle görünmelidir:

In [ ]:
i_daily <- incidence(linelist_clean$date_of_onset) # günlük insidans

plot(i_daily, border = "black")

İnsidans tarihlerinin `i_daily$dates` semptom başlangıcı tarihi hakkında veri sahibi olduğumuz son tarihte (29 Haziran 2014) durduğunu fark edebilirsiniz. Ancak çizgi listesinin yakından incelenmesi, çizgi listesindeki son tarihin (herhangi bir girişin) aslında biraz daha sonra olduğunu gösterir (1 Temmuz 2014). Bunu değiştirmek için `incidence` fonksiyonunda `last_date` argümanını kullanabilirsiniz.

In [ ]:

# son tarihi genişlet:
i_daily <- incidence(linelist_clean$date_of_onset, 
                     last_date = as.Date(max(linelist_clean$date_of_hospitalisation, na.rm = TRUE)))
i_daily
plot(i_daily, border = "black")


Başka bir sorun da günlük insidansa bakarken trendleri yorumlamanın zor olabilmesi, bu nedenle haftalık insidansı `i_weekly`'yi de şu şekilde hesaplayın ve çizin:

In [ ]:
i_weekly <- incidence(linelist_clean$date_of_onset, interval = 7, 
                      last_date = as.Date(max(linelist_clean$date_of_hospitalisation, na.rm = TRUE)))
i_weekly

plot(i_weekly, border = "black")

## Veri ve çıktıları kaydetme

Bu uygulamanın [bölüm 1](./real-time-response-1.html)'inin sonudur. 
[Bölüm 2](./real-time-response-2.html)'ye geçmeden önce, aşağıdaki nesneleri kaydetmeniz gerekecek:

In [ ]:
dir.create(here("data/clean")) # yoksa temiz veri dizini oluştur
saveRDS(i_daily, here("data/clean/i_daily.rds"))
saveRDS(i_weekly, here("data/clean/i_weekly.rds"))
saveRDS(linelist, here("data/clean/linelist.rds"))
saveRDS(linelist_clean, here("data/clean/linelist_clean.rds"))
saveRDS(contacts, here("data/clean/contacts.rds"))